In [3]:
import pandas as pd
import datetime
import gc
from sklearn.ensemble import (
BaggingRegressor, RandomForestRegressor, AdaBoostRegressor)
from sklearn.metrics import mean_squared_error
from technical_indicators import * # import all function
from sklearn.model_selection import TimeSeriesSplit
#import parfit as pf
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import linear_model


In [7]:
# read in our data

#url = "https://raw.githubusercontent.com/SamIlic/Stat479_Project/master/Data/prices.csv"
url = "https://raw.githubusercontent.com/SamIlic/Stat479_Project/master/Data/IBM.csv"
       
df_features = pd.read_csv(url)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 75, saw 4


In [13]:

#df_ORIGINAL = pd.read_csv('IBM.csv')
df_ORIGINAL.drop(['Close'], axis = 1, inplace = True)
df_ORIGINAL.columns = ['Date', 'High', 'Low', 'Open', 'Volume', 'Close']


In [14]:
df_features = df_ORIGINAL
df_features['Date'] = pd.to_datetime(df_features['Date'])
df_features.head()


,Date,High,Low,Open,Volume,Close
0,2002-01-02,121.500000,119.800003,120.599999,6862800,84.677422
1,2002-01-03,124.220001,120.250000,121.500000,8621700,86.182800
2,2002-01-04,125.599999,123.980003,124.050003,8405200,87.534859
3,2002-01-07,126.190002,123.699997,125.000000,5939600,86.454575
4,2002-01-08,125.199997,123.730003,124.250000,5311800,86.907600


In [15]:
### Create Lag Features
# this is exactly what the book does for create_lag_features from chapter 29
def create_lag_features(df, lag_days):
    df_ret = df.copy()
    
    # iterate through the lag days to generate lag values up to lag_days + 1
    for i in range(1,lag_days + 2):
        df_lag = df_ret[['Date', 'Close']].copy()
        # generate dataframe to shift index by i day.
        df_lag['Date'] = df_lag['Date'].shift(-i)
        df_lag.columns = ['Date', 'value_lag' + str(i)]
        # combine the valuelag
        df_ret = pd.merge(df_ret, df_lag, how = 'left', left_on = ['Date'], right_on = ['Date'])
    
    #frees memory
    del df_lag
    
    # calculate today's percentage lag
    df_ret['Today'] = (df_ret['Close'] - df_ret['value_lag1'])/(df_ret['value_lag1']) * 100.0 
    
    # calculate percentage lag
    for i in range(1, lag_days + 1):
        df_ret['lag' + str(i)] = (df_ret['value_lag'+ str(i)] - df_ret['value_lag'+ str(i+1)])/(df_ret['value_lag'+str(i+1)]) * 100.0
    
    # drop unneeded columns which are value_lags
    for i in range(1, lag_days + 2):
        df_ret.drop(['value_lag' + str(i)], axis = 1, inplace = True)
                                                                                                                                                                                                                                                                                                                                     
    return df_ret

In [16]:
df_features = create_lag_features(df_features, 5) # 5 lag features
df_features.head()

,Date,High,Low,Open,Volume,Close,Today,lag1,lag2,lag3,lag4,lag5
0,2002-01-02,121.500000,119.800003,120.599999,6862800,84.677422,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-01-03,124.220001,120.250000,121.500000,8621700,86.182800,1.777781,NaN,NaN,NaN,NaN,NaN
2,2002-01-04,125.599999,123.980003,124.050003,8405200,87.534859,1.568826,1.777781,NaN,NaN,NaN,NaN
3,2002-01-07,126.190002,123.699997,125.000000,5939600,86.454575,-1.234119,1.568826,1.777781,NaN,NaN,NaN
4,2002-01-08,125.199997,123.730003,124.250000,5311800,86.907600,0.524004,-1.234119,1.568826,1.777781,NaN,NaN


In [17]:
# drop earlier data with missing lag features
df_features.dropna(inplace=True)

In [21]:
df_features.head()

,Date,High,Low,Open,Volume,Close,Today,lag1,lag2,lag3,lag4,lag5,STD_14
6,2002-01-10,124.000000,121.419998,123.750000,8926300,85.123428,-1.887755,-0.168376,0.524004,-1.234119,1.568826,1.777781,NaN
7,2002-01-11,122.180000,120.279999,121.500000,4797800,83.848038,-1.498284,-1.887755,-0.168376,0.524004,-1.234119,1.568826,NaN
8,2002-01-14,120.089996,117.099999,120.000000,10026700,82.273003,-1.878440,-1.498284,-1.887755,-0.168376,0.524004,-1.234119,NaN
9,2002-01-15,119.699997,117.720001,118.050003,5647900,82.830559,0.677690,-1.878440,-1.498284,-1.887755,-0.168376,0.524004,NaN
10,2002-01-16,118.750000,117.250000,118.000000,5366300,81.819984,-1.220050,0.677690,-1.878440,-1.498284,-1.887755,-0.168376,NaN


In [22]:
df_features.reset_index(drop = True)
df_features = standard_deviation(df_features, 14)

df_features = relative_strength_index(df_features, 14) # periods
df_features = average_directional_movement_index(df_features, 14, 13) # n, n_ADX
df_features = moving_average(df_features, 21) # periods
df_features = exponential_moving_average(df_features, 21) # periods
df_features = momentum(df_features, 14) # 

df_features = average_true_range(df_features, 14)
df_features = bollinger_bands(df_features, 21)
df_features = ppsr(df_features)
df_features = stochastic_oscillator_k(df_features)
df_features = stochastic_oscillator_d(df_features, 14)
df_features = trix(df_features, 14)
df_features = macd(df_features, 26, 12)
df_features = mass_index(df_features)
df_features = vortex_indicator(df_features, 14)

df_features = kst_oscillator(df_features, 10, 10, 10, 15, 10, 15, 20, 30)
df_features = true_strength_index(df_features, 25, 13)

#df_features = accumulation_distribution(df_features, 14) # Causes Problems, apparently
df_features = chaikin_oscillator(df_features)
df_features = money_flow_index(df_features, 14)
df_features = on_balance_volume(df_features, 14)
df_features = force_index(df_features, 14)
df_features = ease_of_movement(df_features, 14)
df_features = commodity_channel_index(df_features, 14)
df_features = keltner_channel(df_features, 14)
df_features = ultimate_oscillator(df_features)
df_features = donchian_channel(df_features, 14)
    
# drop earlier data with missing lag features
df_features.dropna(inplace=True)
df_features = df_features.reset_index(drop = True)

ValueError: columns overlap but no suffix specified: Index(['STD_14'], dtype='object')